<a href="https://colab.research.google.com/github/MilesZoltak/Google_My_Maps/blob/master/HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 68kB/s 
     |████████████████████████████████| 204kB 41.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=a93c9b75f3b6d7f363cadb6f6e494a4a0ddefc32e41524abbb5a7af9feb87bc7
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext
#initialize spark
spark

# Go crazy!

### Question 1

In [ ]:
fRDD = sc.textFile("soc-LiveJournal1Adj.txt")
fDF = spark.read.text("soc-LiveJournal1Adj.txt")
m1 = fRDD.map(lambda line: (line.split("\t")[0], line.split("\t")[1].split(",")))

for n in range(10):
  secondaries_n_u = m1.filter(lambda x: x[1].count(str(n))).flatMap(lambda y: y[1])
  extras = sc.parallelize(m1.lookup(str(n))[0])
  number = sc.parallelize([str(n)])
  secondaries_n = secondaries_n_u.subtract(extras).subtract(number)
  mapped = secondaries_n.map(lambda x: (x, 1))
  reduced = mapped.reduceByKey(lambda a, b: a+b).sortBy(lambda r: -r[1])
  print(reduced.take(10))

AnalysisException: ignored

In [ ]:
fRDD = sc.textFile("soc-LiveJournal1Adj.txt")
m1 = fRDD.map(lambda line: (line.split("\t")[0], line.split("\t")[1].split(",")))

justUsers = m1.map(lambda user: user[0])
justFriends = m1.map(lambda user: user[1])
sorted = justUsers.map(lambda x: int(x)).sortBy(lambda r: r)
lst = sorted.take(sorted.count())

file = open("friendSuggestions.txt", "w")
# ids = [11, 924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]
for i in lst:
  n = str(i)
  mapped = justFriends.filter(lambda x: x.count(n)).flatMap(lambda y: y)
  number = sc.parallelize([n])
  primaries = sc.parallelize(m1.lookup(n)[0])
  mapped = mapped.subtract(number).subtract(primaries).map(lambda z: (int(z), 1))

  reduced = mapped.reduceByKey(lambda a, b: a+b).sortBy(lambda r: r[0]).sortBy(lambda r: -r[1])
  ln = n + "\t"
  for j in reduced.take(10):
    ln += str(j[0]) + ","
  ln += "\n"
  if (i % 100 == 0):
    print("Processed " + n + " rows.  That's " + str(i/49999 * 100) + "%, yay!")
  file.write(ln)
file.close()

### Question 2

In [ ]:
from pyspark.ml.fpm import FPGrowth
browsing = sc.textFile("browsing.txt")

# track item counts using MapReduce
# mapped = browsing.flatMap(lambda x: x.split()).map(lambda y: (y, 1))
# reduced = mapped.reduceByKey(lambda a, b: a + b).sortBy(lambda r: -r[1]).filter(lambda x: x[1] >= 100)
# freq = reduced.map(lambda x: x[0])
# freq = freq.take(freq.count())

#track pairs where both elements are frequent in a given basket
# pairs = browsing.map(lambda lst: [(a, b) for idx, a in enumerate(lst.split()) for b in lst.split()[idx + 1:]]).flatMap(lambda y: y)
# pairs = pairs.filter(lambda pair: freq.count(pair[0]) and freq.count(pair[1]))
# pairs = pairs.map(lambda x: (x[0], x[1]) if x[0] < x[1] else (x[1], x[0]))  # eliminates issues of (a, b) vs (b, a), since they're the same

triplets = browsing.map(lambda lst: [[x[0], y[0], y[1]] for jdx, x in enumerate([(a, b) for idx, a in enumerate(lst.split()) for b in lst.split()[idx + 1:]]) for y in [(a, b) for idx, a in enumerate(lst.split()) for b in lst.split()[idx + 1:]][jdx + 1:]]).flatMap(lambda y: y)
triplets = triplets.filter(lambda x: x[0] != x[1] and x[0] != x[2])
mapped_ts = triplets.map(lambda x: ((sorted(x)[0], sorted(x)[1], sorted(x)[2]), 1))
reduced_ts = mapped_ts.reduceByKey(lambda a, b: a + b).sortBy(lambda r: -r[1]).filter(lambda z: z[1] >= 100)
reduced_ts = reduced_ts.map(lambda x: ([x[0][0], x[0][1], x[0][2]], x[1]))
count = reduced_ts.count()
df = reduced_ts.toDF()
df.show()
ms = 100 / count
fpGrowth = FPGrowth(itemsCol="_2", minSupport=ms, minConfidence=0.0)
model = fpGrowth.fit(df)
freqs = model.freqItemsets.rdd.sortBy(lambda r: -r[1])
freqs.take(20)
# rfts = reduced_ts.take(reduced_ts.count())
# count = len(rfts)
# conf_scores = []
# iter = 0
# for rft in rfts:
#   if (iter % 500 == 0): print(str(iter) + " out of " + str(count) + ", " + str(iter/count * 100) + "% done")
#   conf_xy = browsing.filter(lambda x: x.count(rft[0][0]) and x.count(rft[0][1])).count() / browsing.filter(lambda x: x.count(rft[0][0])).count()
#   conf_xz = browsing.filter(lambda x: x.count(rft[0][0]) and x.count(rft[0][2])).count() / browsing.filter(lambda x: x.count(rft[0][0])).count()
#   conf_yz = browsing.filter(lambda x: x.count(rft[0][1]) and x.count(rft[0][2])).count() / browsing.filter(lambda x: x.count(rft[0][1])).count()
#   conf_scores.append((rft[0][0] + " --> " + rft[0][1], conf_xy))
#   conf_scores.append((rft[0][0] + " --> " + rft[0][2], conf_xz))
#   conf_scores.append((rft[0][1] + " --> " + rft[0][2], conf_yz))
#   iter += 1
# scores = sc.parallelize(conf_scores).sortBy(lambda r: -r[1])
# scores.take(25)
# mapped_ps = pairs.map(lambda x: (x, 1))
# reduced_ps = mapped_ps.reduceByKey(lambda a, b: a + b).sortBy(lambda r: -r[1])
# filtered_ps = reduced_ps.filter(lambda z: z[1] >= 100)
# #for each reduced pair, calculate conf(a -> b) and conf(b -> a)
#   # that is (for a->b) where (#lines where (a,b) are present / number of lines)/(# of lines where a is present / num of lines)
# rfps = filtered_ps.take(filtered_ps.count())
# conf_scores = []
# for rfp in rfps:
#   conf_ab = browsing.filter(lambda x: x.count(rfp[0][0]) and x.count(rfp[0][1])).count() / browsing.filter(lambda x: x.count(rfp[0][0])).count()
#   conf_scores.append((rfp[0][0] + "--> " + rfp[0][1], conf_ab))
#   conf_ba = browsing.filter(lambda x: x.count(rfp[0][0]) and x.count(rfp[0][1])).count() / browsing.filter(lambda x: x.count(rfp[0][1])).count()
#   conf_scores.append((rfp[0][1] + "--> " + rfp[0][0], conf_ba))

# scores = sc.parallelize(conf_scores).sortBy(lambda r: -r[1])
# scores.take(15)

+--------------------+----+
|                  _1|  _2|
+--------------------+----+
|[DAI75645, FRO402...|8183|
|[DAI62779, FRO402...|7089|
|[DAI62779, ELE929...|6405|
|[DAI62779, DAI756...|6283|
|[FRO40251, GRO850...|6211|
|[DAI62779, ELE174...|5864|
|[DAI62779, DAI756...|5779|
|[DAI62779, FRO402...|5343|
|[DAI75645, FRO402...|5076|
|[ELE17451, FRO402...|5053|
|[DAI62779, ELE174...|4941|
|[DAI62779, FRO402...|4723|
|[FRO40251, FRO924...|4505|
|[DAI75645, FRO402...|4376|
|[DAI75645, ELE174...|4296|
|[DAI62779, ELE174...|4199|
|[DAI62779, ELE321...|4174|
|[DAI62779, DAI756...|4167|
|[DAI62779, DAI853...|4159|
|[DAI43223, DAI627...|4001|
+--------------------+----+
only showing top 20 rows



IllegalArgumentException: ignored